In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Custom layer

In [36]:
class GetEmbeddings(tf.keras.layers.Layer):
    def __init__(self, num_buckets=None, embedding_size=None,*args, **kwargs):
        super(GetEmbeddings, self).__init__(*args, **kwargs)

        #self.shape = compute_output_shape
    
    def build(self, input_shape):
        self.embedding = tf.constant([[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]) # Embedding matrix
    
    def call(self, input):
        user = input[0]
        movie = input[1]
        # Expand user to get it to a matrix of 1 and zeros instead
        user = tf.expand_dims(user, 2)
        user = tf.tile(user, [1,1, 4])
        # check equality of array in t1 for each array in t2 element by element. This is possible because the equal function supports broadcasting. 
        equal =  tf.math.equal(tf.constant([1,1,1,1]), user)
        # checking if all elements from  t1 match for all elements of some array in t2
        equal_all = tf.reduce_all(equal, axis=2)
        user_embeddings = self.embedding * user 
        # Get embeddings for the users based upon the movies he/she watched
        # Expand the movie first
        movies = tf.expand_dims(movie,2)
        movies = tf.tile(movies, [1,1, 4])
        # Get the embeddings
        movie_embeddings = self.embedding * movies
        # Only want the movies with values
        equal_movie =  tf.math.not_equal(tf.constant([0,0,0,0]), movie_embeddings)
        equal_all_movie = tf.reduce_all(equal, axis=1)
        # Get the indicies where we have a movie
        indices = tf.where(equal_movie)
        # In order to to the pointwise multiplication I need to have the same shape so expand
        out = tf.reshape(tf.gather_nd(movie_embeddings,indices),shape=(2,1,4))
        out = tf.tile(out,multiples=[1,4,1])
        # Point wise multiplication, each movie the user watched multiplied with this movie
        output = tf.math.multiply(out,user_embeddings)
        lenght =  tf.reduce_sum(tf.cast(equal_all, tf.float32),axis=1)
        mean = tf.reduce_sum(tf.cast(output,dtype=tf.float32),axis=1) / tf.reshape(lenght, (-1, 1))
        return mean

## Custom layer with dense layer connected

In [37]:
my_seq = tf.keras.Sequential([GetEmbeddings(),
                             tf.keras.layers.Dense(10)])
my_seq([user,movies])

<tf.Tensor: id=391, shape=(2, 10), dtype=float32, numpy=
array([[-3.894199  ,  3.2690303 ,  0.59810704,  3.0813684 ,  4.522706  ,
         1.7911937 ,  0.41366446, -2.7639642 , -3.5591567 , -3.6254656 ],
       [-6.2307177 ,  5.2304487 ,  0.95697117,  4.930189  ,  7.236329  ,
         2.86591   ,  0.6618632 , -4.4223423 , -5.6946507 , -5.800745  ]],
      dtype=float32)>